# Rio De Janero

![Alt Text](https://media.tenor.com/oW_vzeDVjOcAAAAC/travel-rio-de-janeiro.gif)

In [1]:
import pandas as pd
import glob
import os

In [8]:
olist_customers_dataset = pd.read_csv("../Data/olist_customers_dataset.csv")
olist_geolocation_dataset = pd.read_csv("../Data/olist_geolocation_dataset.csv")
olist_order_items_dataset = pd.read_csv("../Data/olist_order_items_dataset.csv")
olist_order_payments_dataset = pd.read_csv("../Data/olist_order_payments_dataset.csv")
olist_order_reviews_dataset = pd.read_csv("../Data/olist_order_reviews_dataset.csv")
olist_orders_dataset = pd.read_csv("../Data/olist_orders_dataset.csv")
olist_products_dataset = pd.read_csv("../Data/olist_products_dataset.csv")
olist_sellers_dataset = pd.read_csv("../Data/olist_sellers_dataset.csv")
product_category_name_translation = pd.read_csv("../Data/product_category_name_translation.csv")